# Система распознавания на Scikit-learn

Готовый практический прототип для курсовой: загрузка данных, предобработка, обучение **Logistic Regression**, **SVM (RBF)** и **Random Forest** через `GridSearchCV`, оценка метрик и визуализации матриц ошибок. 

**Как запускать:**
1. Запустите все ячейки сверху вниз.
2. По умолчанию используется `sklearn.datasets.load_digits` (8x8). Для MNIST (28x28) включите флаг `USE_MNIST = True` в секции ниже.
3. Результаты (отчёты, матрицы ошибок) появятся в конце, лучшие модели сохраняются в `artifacts/`.


In [ ]:
# Библиотеки
import os, json, time
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import joblib

SEED = 42
np.random.seed(SEED)

# Папки для артефактов
ART_DIR = Path('artifacts'); ART_DIR.mkdir(exist_ok=True)

# Можно переключить на полноразмерный MNIST, если среда его предоставляет
USE_MNIST = False  # True -> использовать keras.datasets.mnist


In [ ]:
# Загрузка данных
if USE_MNIST:
    try:
        from tensorflow.keras.datasets import mnist
        (X_train_raw, y_train), (X_test_raw, y_test) = mnist.load_data()
        # Нормализация [0,1] и сплющивание 28x28 -> 784
        X_train = (X_train_raw.reshape((X_train_raw.shape[0], -1)).astype('float32')) / 255.0
        X_test  = (X_test_raw.reshape((X_test_raw.shape[0], -1)).astype('float32')) / 255.0
        print('MNIST loaded:', X_train.shape, X_test.shape)
    except Exception as e:
        print('Не удалось загрузить MNIST через keras, используем sklearn digits. Ошибка:', e)
        data = load_digits()
        X = data.data.astype('float32') / 16.0
        y = data.target
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=SEED, stratify=y
        )
else:
    data = load_digits()
    X = data.data.astype('float32') / 16.0
    y = data.target
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=SEED, stratify=y
    )
print(f"Train: {X_train.shape}, Test: {X_test.shape}")


In [ ]:
# Визуализация примеров
def show_examples(X_like, y_like, n=10, title='Примеры изображений'):
    side = int(np.sqrt(X_like.shape[1])) if X_like.ndim == 2 else X_like.shape[1]
    plt.figure(figsize=(10, 2))
    for i in range(n):
        plt.subplot(1, n, i+1)
        if X_like.ndim == 2:
            plt.imshow(X_like[i].reshape(side, side), cmap='gray')
        else:
            plt.imshow(X_like[i], cmap='gray')
        plt.title(int(y_like[i]))
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

if not USE_MNIST:
    show_examples(data.images.reshape(len(data.images), -1), y[:10], n=10, title='Digits: примеры')


In [ ]:
# Модели + гиперпараметры
pipelines = {
    'LogReg': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=10000, solver='liblinear', random_state=SEED))
    ]),
    'SVM_RBF': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', SVC(probability=True, random_state=SEED))
    ]),
    'RF': Pipeline([
        ('clf', RandomForestClassifier(random_state=SEED))
    ])
}

param_grids = {
    'LogReg': {
        'clf__C': [0.1, 1, 10]
    },
    'SVM_RBF': {
        'clf__C': [0.1, 1, 10],
        'clf__gamma': [0.001, 0.01, 0.1]
    },
    'RF': {
        'clf__n_estimators': [100, 200],
        'clf__max_depth': [None, 10, 20]
    }
}


In [ ]:
# Обучение с GridSearchCV
results = {}
for name, pipe in pipelines.items():
    print(f"\n=== Обучение {name} ===")
    grid = GridSearchCV(
        estimator=pipe,
        param_grid=param_grids[name],
        cv=5,
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )
    grid.fit(X_train, y_train)
    best = grid.best_estimator_
    y_pred = best.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print('Лучшая конфигурация:', grid.best_params_)
    print('CV accuracy:', round(grid.best_score_, 4))
    print('Test accuracy:', round(acc, 4))
    
    # Отчёт и матрица ошибок
    report = classification_report(y_test, y_pred, output_dict=False)
    print(report)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues')
    plt.title(f'Матрица ошибок: {name}')
    plt.show()
    
    # Сохраняем модель и артефакты
    joblib.dump(best, ART_DIR / f'best_{name}.joblib')
    np.savetxt(ART_DIR / f'cm_{name}.csv', cm, fmt='%d', delimiter=',')
    results[name] = {
        'best_params': grid.best_params_,
        'cv_accuracy': float(grid.best_score_),
        'test_accuracy': float(acc)
    }

with open(ART_DIR / 'summary.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
print("\nСводка:", json.dumps(results, ensure_ascii=False, indent=2))


## Итоги
- Сравнили **Logistic Regression**, **SVM (RBF)** и **Random Forest** с кросс-валидацией и перебором гиперпараметров.
- Сохранили лучшие модели и матрицы ошибок в `artifacts/`.
- Для переноса на другой набор (MNIST) выставьте `USE_MNIST=True` (если доступен `keras.datasets`).
